# Getting Started with `FRAME`

This notebook illustrates a complete example of how to run `FRAME` on a dataset of North American gray wolves published in [Schweizer et al. 2015](https://onlinelibrary.wiley.com/doi/full/10.1111/mec.13364?casa_token=idW0quVPOU0AAAAA%3Ao_ll85b8rDbnW3GtgVeeBUB4oDepm9hQW3Y445HI84LC5itXsiH9dGO-QYGPMsuz0b_7eNkRp8Mf6tlW).

## Import packages

First we import the required packages and `FRAME`:

In [3]:
# base
import numpy as np
import pkg_resources
from sklearn.impute import SimpleImputer
from pandas_plink import read_plink

# visualization
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# FRAME
from FRAME.utils import prepare_graph_inputs
from FRAME import SpatialDiGraph, Vis
from FRAME.cross_validation import run_cv

ModuleNotFoundError: No module named 'FRAME.utils'

## Prepare the data

Note we have packaged this example dataset in the `FRAME` package and use the `pkg_resources` package to find the path of those files:

In [4]:
data_path = pkg_resources.resource_filename("FRAME", "data/")

TypeError: expected str, bytes or os.PathLike object, not NoneType

Next we read the `plink` formatted genotype data and impute any missing SNPs with the mean at each SNP:

In [3]:
# read the genotype data and mean impute missing data
(bim, fam, G) = read_plink("{}/wolvesadmix".format(data_path))
imp = SimpleImputer(missing_values=np.nan, strategy="mean")
genotypes = imp.fit_transform((np.array(G)).T)

print("n_samples={}, n_snps={}".format(genotypes.shape[0], genotypes.shape[1]))

Mapping files: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 45.28it/s]

n_samples=111, n_snps=17729


As we can see we have 111 samples and 17,729 SNPs. For preparing the graph inputs to run `FRAME` you have two options:

* Prepare your own input files
* Use the `FRAME` function `prepare_graph_inputs` which intersects a discrete global grid (DGG) with the sample range

We'll show off the later option. We read the sample coordinates, coordinates of the outer polygon that defines the habitat of the sample and a discrete global grid file which has laid down a triangular grid that is uniformly spaced on earth. We then intersect this global grid with the outer file to define the graph that we use to optimize:

In [5]:
%%time
# setup graph
coord = np.loadtxt("{}/wolvesadmix.coord".format(data_path))  # sample coordinates
outer = np.loadtxt("{}/wolvesadmix.outer".format(data_path))  # outer coordinates
grid_path = "{}/grid250.shp".format(data_path)  # path to discrete global grid

# graph input files
outer, edges, grid, _ = prepare_graph_inputs(coord=coord, 
                                             ggrid=grid_path,
                                             translated=True, 
                                             buffer=0,
                                             outer=outer)

NameError: name 'data_path' is not defined

## Setup the `SpatialDiGraph` object

We then setup the `SpatialDiGraph` object which is the core workhorse of `FRAME`. `SpatialDiGraph` specifies the graph, allele frequency data, and runs the optimizers to fit the edge weights of the digraph:

In [7]:
%%time
sp_graph = SpatialDiGraph(genotypes, coord, grid, edges)

NameError: name 'SpatialDiGraph' is not defined

This might take a few minutes to construct at first b/c it initializing a number of graph matrices that are slow to build. First, before any fitting we'll visualize the graph and samples. Lets setup the projection we'll be using for this dataset:

In [9]:
projection = ccrs.EquidistantConic(central_longitude=-108.842926, central_latitude=66.037547)

Now lets make a map of the sample coordinates, graph and observed nodes:

In [8]:
fig = plt.figure(dpi=300)
ax = fig.add_subplot(1, 1, 1, projection=projection)  
v = Vis(ax, sp_digraph, projection=projection, edge_width=.5, 
        edge_alpha=1, edge_zorder=100, sample_pt_size=10, 
        obs_node_size=7.5, sample_pt_color="black", 
        cbar_font_size=10)
v.draw_map()
v.draw_samples()
v.draw_edges(use_weights=False)
v.draw_obs_nodes(use_ids=False)

NameError: name 'projection' is not defined

<Figure size 1920x1440 with 0 Axes>

## Run cross validation

Next we perform 10-fold cross-validation over $\lambda$ through a two-stage grid search. In our CV we hold out individual observed nodes on the digraph, predict allele frequencies at the held-out node under our fitted model, and use the $\ell_2$ distance between the fitted and predicted allele frequencies as our CV metric to select models. We first do a rough search and determine the range of the fine grid search:

In [11]:
lamb_warmup = 1e3

lamb_grid = np.geomspace(1e-3, 1e3,13)[::-1]

cv,node_train_idxs=run_cv(sp_digraph,
                          lamb_grid,
                          lamb_warmup=lamb_warmup,
                          n_folds=10,
                          factr=1e10,
                          random_state=500,
                          outer_verbose=True,
                          inner_verbose=False,)

if np.argmin(cv)==0:
   lamb_grid_fine=np.geomspace(lamb_grid[0],lamb_grid[1],5)[::-1]

elif np.argmin(cv)==12:
     lamb_grid_fine=np.geomspace(lamb_grid[11],lamb_grid[12], 5)[::-1]
     
else:
    lamb_grid_fine=np.geomspace(lamb_grid[np.argmin(cv)-1],lamb_grid[np.argmin(cv)+1], 5)[::-1]

NameError: name 'np' is not defined

We then do a fine grid search and pick the optimal $\lambda$ value:

In [16]:
cv_fine,node_train_idxs_fine=run_cv(sp_digraph,
                                    lamb_grid_fine,
                                    lamb_warmup=lamb_warmup,
                                    n_folds=10,
                                    factr=1e10,
                                    random_state=500,
                                    outer_verbose=True,
                                    inner_verbose=False,
                                    node_train_idxs=node_train_idxs)

lamb_opt=lamb_grid_fine[np.argmin(cv_fine)]
lamb_opt=float("{:.3g}".format(lamb_opt))

NameError: name 'run_cv' is not defined

The CV error can be ploted as following:

In [19]:
plt.figure(figsize=(8, 6))
plt.plot(np.log10(lamb_grid), cv, 'bo')  
plt.plot(np.log10(lamb_grid_fine), cv_fine, 'bo')  
plt.xlabel(r"$\mathrm{log}_{10}(\mathrm{\lambda})$")
plt.ylabel('CV Error')

NameError: name 'plt' is not defined

## Fit `FRAME`

Next we fit the `FRAME` model with the optimal $\lambda$. To be consistent with the cross validation step, we use the warm up fitting as the intial value and then fit with stricter converging criteria: 

In [17]:
%%time
sp_digraph.fit(lamb=lamb_warmup, factr=1e10)
logm = np.log(sp_digraph.m)
logc = np.log(sp_digraph.c)

sp_digraph.fit(lamb=lamb_opt,
               factr=1e7,
               logm_init=logm,
               logc_init=logc,
               )

NameError: name 'sp_digraph' is not defined

## Visualize the results

Now we can visualize the results:

In [20]:
fig, axs= plt.subplots(2, 4, figsize=(15, 6), dpi=300,
                        subplot_kw={'projection': projection})

v = Vis(axs[0,0], sp_digraph, projection=projection, edge_width=0.5,
        edge_alpha=1, edge_zorder=100, sample_pt_size=20,
        obs_node_size=5, sample_pt_color="black",
        cbar_font_size=10, cbar_ticklabelsize=8, mutation_scale=3)

v.digraph_wrapper(axs, node_scale=[5, 5, 5])

NameError: name 'plt' is not defined

The `digraph_wrapper` function shows all the representations and attributes in a single figure. If we want to draw them seperately, we can use `draw_migration_rates` function and `draw_attributes` function, with a specification of the graph mode (`Base`, `Full`, `Difference`, `Summary`) and attributes (`Sample Size and Position`, `Heterozygosity`,`Stationary Distribution`,`Coalescent Rate`). For example, we can show the difference graph and coalescent rate through:

In [1]:
ax= fig.add_subplot(1, 1, 1, projection=projection)
v.draw_migration_rates(ax,
                       mode='Difference',
                       draw_map=True,
                       draw_nodes=True,)

ax= fig.add_subplot(1, 1, 1, projection=projection) 
v.draw_attributes(ax,
                  node_scale=5,
                  attribute='Coalescent Rate',
                  draw_map=True,)

NameError: name 'fig' is not defined

We can also compare the fitted genetic distances and the observed genetic distances:

In [2]:
digraphstats = Digraphstats(sp_digraph)

fig, axs = plt.subplots(1, 3, figsize=(18, 5), dpi=300)
digraphstats.fitting_wrapper(axs)
plt.show()

NameError: name 'Digraphstats' is not defined

The fitting_wrapper function combines the regression of distances, z-score distribution and the heatmap, which can also be drawn seperately:

In [3]:
fig, ax = plt.subplots(1, 1, figsize=(6, 5), dpi=300)
digraphstats.distance_regression(ax)
 
fig, ax = plt.subplots(1, 1, figsize=(6, 5), dpi=300)
digraphstats.z_score_distribution(ax)
        
fig, ax = plt.subplots(1, 1, figsize=(6, 5), dpi=300)   
digraphstats.draw_heatmap(ax)

NameError: name 'plt' is not defined

## Detect Outliers

Great ... now we finish our first `FRAME` analysis! We essentially provided these results interactively! For more interpretation of these  figures and method please see our pre-print :)